In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')

In [3]:
train_set=data.fillna(0.0)
train_set

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0,0.106643,3.594370,132.8040,3.184280,0.081971,1.18859,3.732380,2.266270,2.099590,...,1.098620,0.013331,-0.011715,0.052759,0.065400,4.211250,1.978770,0.085974,0.240496,0
1,1,0.125021,1.673360,76.5336,3.378250,0.099400,5.09366,1.275620,-0.471318,4.545940,...,3.460170,0.017054,0.124863,0.154064,0.606848,-0.267928,2.577860,-0.020877,0.024719,0
2,2,0.036330,1.497470,233.5460,2.194350,0.026914,3.12694,5.056870,3.849460,1.801870,...,4.883000,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.141120,0.124464,0.148209,0
3,3,-0.014077,0.246000,779.9670,1.890640,0.006948,1.53112,2.698000,4.517330,4.503320,...,3.474390,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.968600,0.040017,0.044873,0
4,4,-0.003259,3.715420,156.1280,2.147720,0.018284,2.09859,4.154920,-0.038236,3.371450,...,1.910590,-0.042943,0.105616,0.125072,0.037509,1.043790,1.074810,-0.012819,0.072798,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,599995,0.043008,1.640550,1375.8900,1.087030,0.839580,1.69362,3.843520,1.108250,1.505390,...,1.552280,0.080317,0.017643,0.225133,0.073179,4.436510,1.739330,0.049038,0.065804,1
599996,599996,0.137048,4.826860,-99.6478,0.985289,0.037712,3.30370,2.471790,5.942130,0.875148,...,1.345210,0.092457,0.081926,0.234041,0.041383,3.873990,1.870100,0.162313,0.085662,0
599997,599997,1.316220,0.502062,66.2360,2.233020,0.060038,3.77371,2.655340,3.155890,1.055020,...,2.181150,0.032708,0.109668,0.027466,0.061931,-0.383329,0.922113,0.084864,0.052635,0
599998,599998,0.511918,3.827470,777.8720,0.285914,0.520157,2.76285,2.522200,1.561360,1.971140,...,0.366702,0.070943,0.071452,-0.038071,0.038829,-0.122888,3.107790,0.061333,0.004179,1


In [4]:
X=train_set.iloc[:,:-1].values
Y=train_set.iloc[:,-1].values

In [5]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler
tr=StandardScaler()
X=tr.fit_transform(X)

In [7]:
import tensorflow as tf

In [8]:
tf.__version__

'2.6.0'

In [9]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
es=EarlyStopping( monitor='val_loss', 
    min_delta=0, 
    patience=16, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True)
plateau=ReduceLROnPlateau( monitor='val_loss',factor=0.2, 
    patience=5,
    mode='min')

In [10]:
from keras.backend import sigmoid
def swish(x,beta=1):
  return (x*sigmoid(beta*x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish':Activation(swish)})

In [11]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout, Input
def get_model():
  
  inp = Input(shape=X.shape[1], name='input')
  h=Dense(units=128,activation='swish')(inp)
  h=Dropout(0.25)(h)

  h=Dense(units=64,activation='swish')(h)
  h=Dropout(0.25)(h)

  h=Dense(units=32,activation='swish')(h)
  h=Dropout(0.25)(h)

  h=Dense(units=1,activation='sigmoid')(h)

  model=Model(inputs=inp,outputs=h)

  model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-3),
        metrics=['AUC']
    )
  
  return model

In [12]:
test_set=pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
test_set

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,600000,0.003229,4.838660,585.5290,2.282910,0.713180,3.907830,0.480696,1.482270,4.891810,...,0.112910,1.073550,0.122149,0.086330,0.036010,0.010619,0.290343,1.898200,0.131533,0.012047
1,600001,0.008602,0.505536,-100.0990,3.012670,0.027199,1.194610,5.036620,2.517440,4.553890,...,-0.020214,2.622340,0.123307,0.033063,0.123059,0.005771,-0.392923,3.689640,0.047418,0.120015
2,600002,1.461000,2.437260,-112.9640,3.541230,0.752338,4.338310,1.648080,4.699910,1.950250,...,-0.011036,2.030180,-0.000426,0.084091,0.123605,0.499554,4.054650,3.330670,0.108843,0.064687
3,600003,0.140556,3.085610,179.4510,0.573945,0.057342,2.216790,1.623480,0.526174,1.542540,...,0.050117,0.221613,0.045298,0.129966,0.004015,0.018279,2.696580,-0.533491,0.052524,0.011058
4,600004,0.128876,5.199760,107.4660,-0.497149,0.080220,0.458121,0.629839,5.240460,-0.232279,...,0.058860,2.660430,0.135425,0.036481,0.093912,0.056315,1.110710,3.584470,0.145319,-0.050393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539995,1139995,0.431599,1.507560,24.2698,2.928480,2.788830,5.152330,2.779980,0.816389,4.791560,...,0.083219,2.545860,0.132701,0.040109,0.121019,0.024841,3.182610,1.252730,0.059733,0.029033
539996,1139996,0.069713,2.355480,-128.7550,2.721580,1.256300,4.248220,2.014550,2.207120,3.020260,...,0.077851,4.019700,0.165319,0.065945,0.081351,-0.022179,4.512380,2.812880,0.016573,0.079498
539997,1139997,0.385075,2.528890,-63.9854,0.975396,0.043852,0.829423,2.014210,1.509500,2.027590,...,0.060691,3.604950,0.092720,0.067277,0.113736,0.132187,0.567747,-1.112850,0.776967,0.123728
539998,1139998,1.846240,3.415350,26.8476,-0.120134,0.027113,2.155160,2.529860,2.502250,3.453090,...,-0.012514,0.122062,0.099288,0.043613,0.013528,0.078044,-0.337694,0.436730,0.054561,0.082622


In [13]:
test_set=tr.transform(test_set)

In [14]:
test_set.shape

(540000, 101)

In [15]:
EPOCHS = 100
BATCH_SIZE = 1024
VERBOSE = 0
N_SPLITS = 10
predictions=[]

from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
scores = {fold:None for fold in range(cv.n_splits)}
for fold,(train_idx,test_idx) in enumerate(cv.split(X,Y)):
  x_train,x_test=X[train_idx],X[test_idx]
  y_train,y_test=Y[train_idx],Y[test_idx]

  model=get_model()
  history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=VERBOSE,
        callbacks=[
            es,
            plateau
        ]
    )
    
  scores[fold] = (history.history)

  prediction = model.predict(test_set, batch_size=BATCH_SIZE).reshape(1,-1)[0]
  predictions.append(prediction)

  print(f"Fold {fold+1} || Max Validation AUC: {np.max(scores[fold]['val_auc'])}")


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Fold 1 || Max Validation AUC: 0.7545914649963379
Fold 2 || Max Validation AUC: 0.7542312145233154
Fold 3 || Max Validation AUC: 0.7561343312263489
Fold 4 || Max Validation AUC: 0.7564812898635864
Fold 5 || Max Validation AUC: 0.7574330568313599


In [16]:
predictions

[array([0.73937565, 0.7231837 , 0.7450819 , ..., 0.68432343, 0.75018036,
        0.7288269 ], dtype=float32),
 array([0.7412736 , 0.7493144 , 0.73928344, ..., 0.7033013 , 0.7395533 ,
        0.72719765], dtype=float32),
 array([0.74190354, 0.7229867 , 0.7578287 , ..., 0.7026529 , 0.7325155 ,
        0.6970341 ], dtype=float32),
 array([0.7235539 , 0.73979604, 0.7473176 , ..., 0.65988404, 0.73979807,
        0.7396836 ], dtype=float32),
 array([0.73522985, 0.73526424, 0.75605   , ..., 0.68736535, 0.7300581 ,
        0.7227533 ], dtype=float32)]

In [17]:
sample_submission=pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sample_submission

,id,target
0,600000,0.5
1,600001,0.5
2,600002,0.5
3,600003,0.5
4,600004,0.5
...,...,...
539995,1139995,0.5
539996,1139996,0.5
539997,1139997,0.5
539998,1139998,0.5


In [18]:
predictions

[array([0.73937565, 0.7231837 , 0.7450819 , ..., 0.68432343, 0.75018036,
        0.7288269 ], dtype=float32),
 array([0.7412736 , 0.7493144 , 0.73928344, ..., 0.7033013 , 0.7395533 ,
        0.72719765], dtype=float32),
 array([0.74190354, 0.7229867 , 0.7578287 , ..., 0.7026529 , 0.7325155 ,
        0.6970341 ], dtype=float32),
 array([0.7235539 , 0.73979604, 0.7473176 , ..., 0.65988404, 0.73979807,
        0.7396836 ], dtype=float32),
 array([0.73522985, 0.73526424, 0.75605   , ..., 0.68736535, 0.7300581 ,
        0.7227533 ], dtype=float32)]

In [19]:
c=np.mean(np.column_stack(predictions),axis=1)
c

array([0.73626727, 0.73410904, 0.7491123 , ..., 0.68750536, 0.7384211 ,
       0.7230991 ], dtype=float32)

In [20]:
sample_submission['target'] = np.mean(np.column_stack(predictions), axis=1)
sample_submission.to_csv('./Tps(Nov).csv', index=False)

postprocess_separate(sample_submission)
sample_submission.to_csv('./nn_model(Tps-(Nov)).csv', index=False)

NameError: name 'postprocess_separate' is not defined

In [ ]:
cal=np.mean(np.column_stack(predictions), axis=1)
cal

In [ ]:
value=sample_submission['id']

In [ ]:

output=pd.DataFrame({'id':value,'target':cal})
output.to_csv('submission.csv')
output